# DSCI 511: Data Acquisition and Pre-Processing - Final Project <br> 

# 2. Integration of Findings over Multiple Titles 

In [22]:
from bs4 import BeautifulSoup
import urllib 
import requests
import re
from collections import defaultdict
import pandas as pd
import time

Below function gets the data from the main page which lists multiple titles of sequels (100 titles at each page).

In [23]:
# Get total number of movies 
def get_soup(sequel_url):
    url = sequel_url 
    time.sleep(10)
    response = requests.get(url)
    data = response.text
    soup = BeautifulSoup(data)
    
    return(soup)

<font size="4">**2.1. General Information**<font>

Below function gets the general information of each movie, as appearing in the the list. 

- We have considered only movies which heve released before 2020 because information for newly-released movies or those supposed to be relased in future is not complete. Obviuosly, this cut-off value of year can be changed by the user.


- Few movies don't have information pertinent to Metascore and Gross Budget.  This limitation was also addressed inside the function. 

In [29]:
def scrap_movie_data(sequel_url):
    
    movie_dataframe = pd.DataFrame()
    
    for i in range(1,10):
        
        url = sequel_url + '?sort=date_added,desc&st_dt=&mode=detail&page=' + str(i)
        soup = get_soup(url)
        x = soup.find_all('div',attrs={'class' : 'lister-item-content'})
        for i in x:
            movie = {}
            movie['Title'] = i.find('a').contents[0]
            year = i.find('span', attrs={'class' : 'lister-item-year text-muted unbold'}).text
            if year:
                year = int(''.join(filter(str.isdigit, year)))
            else:
                year = 2021
            if year < 2020:
                movie['Year'] = year
                movie['Length'] = i.find('span', attrs={'class' : "runtime"}).text
                movie['Genre'] =  i.find('span', attrs={'class' : "genre"}).text
                movie['Certificate'] = i.find('span', attrs={'class' : "certificate"}).text.strip()
                movie['Rating'] = i.find('span', attrs={'class' : "ipl-rating-star__rating"}).text
                Metascore = i.find('span', attrs={'class' : "metascore"})
                if Metascore:
                    movie['Metascore'] = Metascore.text.strip()
                else:
                    movie['Metascore'] = None
                movie['Story_line'] = i.find('p',attrs={'class': ""}).text.strip()
                for header in i.find_all('p',attrs={'class': "text-muted text-small"}):
                    if header.find_all('span'):
                        if "Director" in header.text:
                            director_stars = [i.strip() for i in header.text.split('|')]
                            movie['Director'] = [i.strip() for i in director_stars[0].partition(":")[2].split(',')]
                            movie['Stars'] = [i.strip() for i in director_stars[1].partition(":")[2].split(',')]            
                        if "Votes" in header.text:
                            Votes_Gross = [i.strip() for i in header.text.split('|')]
                            movie['Votes'] = Votes_Gross[0].partition(":")[2].strip()
                            if len(Votes_Gross)>1:
                                movie['Gross'] = Votes_Gross[1].partition(":")[2].strip()
                            else:
                                movie['Gross'] = 'Not Available'
                movie['Title_url']=i.find('a')['href']
                movie_dataframe = movie_dataframe.append(pd.DataFrame([movie]),  ignore_index = True)
              
    return(movie_dataframe)

In [15]:
sequel_url = "https://www.imdb.com/list/ls003495084/"

In [30]:
movie_data = scrap_movie_data(sequel_url)
movie_data

,Title,Year,Length,Genre,Certificate,Rating,Metascore,Story_line,Director,Stars,Votes,Gross,Title_url
0,Spider-Man,2002,121 min,"\nAction, Adventure, Sci-Fi",PG-13,7.3,73,"When bitten by a genetically modified spider, ...",[Sam Raimi],"[Tobey Maguire, Kirsten Dunst, Willem Dafoe, J...","669,928",$403.71M,/title/tt0145487/
1,Spider-Man 2,2004,127 min,"\nAction, Adventure, Sci-Fi",PG-13,7.3,83,Peter Parker is beset with troubles in his fai...,[Sam Raimi],"[Tobey Maguire, Kirsten Dunst, Alfred Molina, ...","527,736",$373.59M,/title/tt0316654/
2,Spider-Man 3,2007,139 min,"\nAction, Adventure, Sci-Fi",PG-13,6.2,59,A strange black entity from another world bond...,[Sam Raimi],"[Tobey Maguire, Kirsten Dunst, Topher Grace, T...","485,196",$336.53M,/title/tt0413300/
3,The Matrix,1999,136 min,"\nAction, Sci-Fi",R,8.7,73,A computer hacker learns from mysterious rebel...,"[Lana Wachowski, Lilly Wachowski]","[Keanu Reeves, Laurence Fishburne, Carrie-Anne...","1,613,016",$171.48M,/title/tt0133093/
4,The Matrix Reloaded,2003,138 min,"\nAction, Sci-Fi",R,7.2,62,Neo and his allies race against time before th...,"[Lana Wachowski, Lilly Wachowski]","[Keanu Reeves, Laurence Fishburne, Carrie-Anne...","501,450",$281.49M,/title/tt0234215/
...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,Final Destination 5,2011,92 min,"\nHorror, Thriller",R,5.8,50,Survivors of a suspension-bridge collapse lear...,[Steven Quale],"[Nicholas D'Agosto, Emma Bell, Arlen Escarpeta...","103,591",$42.59M,/title/tt1622979/
896,Underworld,2003,121 min,"\nAction, Fantasy, Thriller",R,7,42,"Selene, a vampire warrior, is entrenched in a ...",[Len Wiseman],"[Kate Beckinsale, Scott Speedman, Shane Brolly...","247,647",$51.97M,/title/tt0320691/
897,Underworld: Evolution,2006,106 min,"\nAction, Fantasy, Sci-Fi",R,6.7,36,"Picking up directly from the previous movie, v...",[Len Wiseman],"[Kate Beckinsale, Scott Speedman, Bill Nighy, ...","186,149",$62.32M,/title/tt0401855/
898,Underworld: Rise of the Lycans,2009,92 min,"\nAction, Fantasy",R,6.6,44,An origins story centered on the centuries-old...,[Patrick Tatopoulos],"[Rhona Mitra, Michael Sheen, Bill Nighy, Steve...","147,789",$45.80M,/title/tt0834001/


<font size="4">**2.2. Detailed Information**<font>

In this section, we are scraping detailed information of each title through navigating to its own link from the list.

- Below is the output for the first 20 movies. A user can decide the number of titles he or she needs detailed information for and automate the scraping procedure as illustrated below.


- Columns can be joined with those of the previuosly created dataframe on common titles.

In [123]:
movie_url = movie_data.iloc[:20,-1]
Title =  movie_data.iloc[:1,0]
title_details = pd.DataFrame()
for i in movie_url:
    m = {}
    title_url = "https://www.imdb.com"+i+ '?ref_=ttls_li_tt'
    title_data = get_soup(title_url)
    title = title_data.find('div', attrs = {'class' : 'title_wrapper'}).contents[1].text
    details = title_data.find('div', attrs = {'id' : 'titleDetails'})
    for header in details.find_all('div', attrs = {'class' : 'txt-block'}):
        if 'Country' in header.text:
            m['Country'] = [i.strip() for i in header.text.partition(":")[2].split(',')]
        if 'Language' in header.text:
            m['Language'] = [i.strip() for i in header.text.partition(":")[2].split(',')]
        if 'Budget' in header.text:
            m['Budget'] = header.text.partition(":")[2].split('\n')[0]
        if 'Opening Weekend' in header.text:
            m['Opening'] = header.text.partition(":")[2].split('\n')[0].strip()
        if 'Gross USA' in header.text:
            m['Gross_USA'] = header.text.partition(":")[2].split('\n')[0].strip()
        if 'Worldwide Gross' in header.text:
            m['Worldwide_Gross'] = header.text.partition(":")[2].split('\n')[0].strip()
    title_details = title_details.append(pd.DataFrame([m]), ignore_index = True)
title_details

,Country,Language,Budget,Opening,Gross_USA,Worldwide_Gross
0,[USA],[English],"$139,000,000","$114,844,116,","$407,022,860","$825,025,036"
1,[USA],[English\n|\nRussian\n|\nChinese],"$200,000,000","$88,156,227,","$373,585,825","$788,976,453"
2,[USA],[English\n|\nFrench],"$258,000,000","$151,116,516,","$336,530,303","$894,983,373"
3,[USA],[English],"$63,000,000","$27,788,331,","$171,479,930","$465,343,787"
4,[USA],[English\n|\nFrench],"$150,000,000","$91,774,413,","$281,576,461","$741,846,459"
5,[USA],[English\n|\nFrench],"$150,000,000","$48,475,154,","$139,313,948","$427,344,031"
6,[New Zealand\n|\nUSA],[English\n|\nSindarin],"$93,000,000","$47,211,490,","$315,544,750","$887,832,826"
7,[New Zealand\n|\nUSA],[English\n|\nSindarin\n|\nOld English],"$94,000,000","$62,007,528,","$342,551,365","$951,208,089"
8,[New Zealand\n|\nUSA],[English\n|\nQuenya\n|\nOld English\n|\nSindarin],"$94,000,000","$72,629,713,","$377,845,905","$1,142,219,401"
9,[USA],[English],"$19,000,000","$11,152,500,","$211,159,762","$388,355,305"
